# 0. Text2SQL Data Pre-Processing
**Step 0:** Process Dataset For Training & Evaluation

In [1]:
pip install fsspec==2024.10.0

In [3]:
!pip cache purge

Files removed: 30


In [1]:
!pip install -q -U datasets

In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.4 MB/s eta 0:00:00


In [3]:
from evaluate import load

### Imports

In [4]:
import pandas as pd
import json
import torch
import os

# Load Methods from Datasets Library
from datasets import load_dataset, Dataset, load_from_disk

### Global Constants

In [5]:
dataset_name = "b-mc2/sql-create-context"

In [7]:
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# GDrive Location for Train/Test Data
DATA_PATH ="/content/drive/MyDrive/Text2SQL/Data/"
DS_DIR = "sql_train_test"
PKL_DIR = "test/"
PKL_FILE ="sql_test.pkl"

# Generating Train/Test Data Parameters
TABLE_NAMES = True # Drops Most of WikiSQL
SIMPLE_INST = False # Complex Prompt
SAMPLE_RATE = 0.1 # Train=90% vs Test=10% Split

### Common Functions

In [9]:
def process_data(dataset_name, sample_rate,
                 table_names=False, simple_inst=False):
  '''Function Returns a DataFrame '''
  # 1. Move data to dataframe
  txt2sql_ds = load_dataset(dataset_name)
  txt2sql_df = pd.DataFrame(txt2sql_ds)
  dsdf = pd.json_normalize(txt2sql_df['train'])


  # 2. Cleanup Steps
  # Dropping all examples where no proper table name is given i.e., Wikisql
  if table_names:
    dsdf = dsdf.loc[~dsdf['answer'].str.contains('FROM table_')]
  # Drop Duplicates
  dsdf.drop_duplicates(inplace=True)
  dsdf.rename(columns={'answer': 'response'}, inplace=True)

  # 3. Simple Instruction
  if simple_inst:
    template = """Below is an instruction that describes a task. \
    Write a response that appropriately completes the request.

    ### Instruction:
    Generate SQL query: {question}, \
    given the following schema: {context}

    ### Response:
    {response}
    ### End"""
  else:
    # change instuction if required
    template = """### Instruction:
    You are a powerful text-to-SQL model. \
    Your job is to answer questions about a database. \
    You are given a question and context regarding one or more tables.

    You must output the SQL query that answers the question.

    ### Input:
    {question}
    ### Context:
    {context}
    ### Response:
    {response}
    ### End"""

  dsdf['text'] = dsdf.apply(template.format_map, axis=1)
  display(dsdf.head(2))
  dataset = Dataset.from_pandas(dsdf).train_test_split(test_size=sample_rate,
                                                       seed=42)
  print('Training Sample:')
  display(pd.DataFrame(dataset["train"]).head(2))
  print('Testing Sample:')
  display(pd.DataFrame(dataset["test"]).head(2))
  return dsdf, dataset

In [10]:
def process_test(ds, col='test', table_names=False, simple_inst=False):
  dsdf = pd.DataFrame(ds[col])

  # 2. Cleanup Steps
  # Dropping all examples where no proper table name is given i.e., Wikisql
  if table_names:
    dsdf = dsdf.loc[~dsdf['response'].str.contains('FROM table_')]
  # Drop Duplicates
  dsdf.drop_duplicates(inplace=True)
  dsdf.drop(columns=['text'], inplace=True)

  # 3. Simple Instruction
  if simple_inst:
    template = """Below is an instruction that describes a task. \
    Write a response that appropriately completes the request.

    ### Instruction:
    Generate SQL query: {question}, \
    given the following schema: {context}

    ### Response:
    """
  else:
    # change instuction if required
    template = """### Instruction:
    You are a powerful text-to-SQL model. \
    Your job is to answer questions about a database. \
    You are given a question and context regarding one or more tables.

    You must output the SQL query that answers the question.

    ### Input:
    {question}
    ### Context:
    {context}
    ### Response:
    """

  dsdf['text'] = dsdf.apply(template.format_map, axis=1)
  display(dsdf.head(2))
  return dsdf

### Load and Store Process Dataset
- Stored as HF Dataset

In [11]:
dsdf, dataset = process_data(dataset_name, sample_rate=SAMPLE_RATE,
                             table_names=TABLE_NAMES, simple_inst=SIMPLE_INST)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.43k [00:00<?, ?B/s]

sql_create_context_v4.json:   0%|          | 0.00/21.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

,response,question,context,text
0,SELECT COUNT(*) FROM head WHERE age > 56,How many heads of the departments are older th...,CREATE TABLE head (age INTEGER),### Instruction:\n You are a powerful text-...
1,"SELECT name, born_state, age FROM head ORDER B...","List the name, born state and age of the heads...","CREATE TABLE head (name VARCHAR, born_state VA...",### Instruction:\n You are a powerful text-...


Training Sample:


,response,question,context,text,__index_level_0__
0,SELECT Nationality FROM people GROUP BY Nation...,What are the nationalities that are shared by ...,CREATE TABLE people (Nationality VARCHAR),### Instruction:\n You are a powerful text-...,4326
1,SELECT T2.balance FROM accounts AS T1 JOIN che...,What is the checking balance of the account wh...,"CREATE TABLE checking (balance VARCHAR, custid...",### Instruction:\n You are a powerful text-...,1034


Testing Sample:


,response,question,context,text,__index_level_0__
0,"SELECT T2.name, COUNT(*) FROM race AS T1 JOIN ...",Show the name of track and the number of races...,"CREATE TABLE track (name VARCHAR, track_id VAR...",### Instruction:\n You are a powerful text-...,429
1,"SELECT T3.Shop_Name, T2.Carrier FROM stock AS ...",Show names of shops and the carriers of device...,"CREATE TABLE shop (Shop_Name VARCHAR, Shop_ID ...",### Instruction:\n You are a powerful text-...,2907


In [12]:
dataset.save_to_disk(DATA_PATH + DS_DIR)

Saving the dataset (0/1 shards):   0%|          | 0/4086 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/454 [00:00<?, ? examples/s]

In [13]:
ds2=load_from_disk(DATA_PATH + DS_DIR)

#### Check Stored Dataset

In [14]:
display(dataset['train'])
display(dataset['test'])

Dataset({
    features: ['response', 'question', 'context', 'text', '__index_level_0__'],
    num_rows: 4086
})

Dataset({
    features: ['response', 'question', 'context', 'text', '__index_level_0__'],
    num_rows: 454
})

In [15]:
display(ds2['train'])
display(ds2['test'])

Dataset({
    features: ['response', 'question', 'context', 'text', '__index_level_0__'],
    num_rows: 4086
})

Dataset({
    features: ['response', 'question', 'context', 'text', '__index_level_0__'],
    num_rows: 454
})

### Save Test Pandas DataFrame

In [16]:
test_df = process_test(dataset, col='test', table_names=TABLE_NAMES,
                       simple_inst=SIMPLE_INST)
display(test_df['text'][4])

,response,question,context,__index_level_0__,text
0,"SELECT T2.name, COUNT(*) FROM race AS T1 JOIN ...",Show the name of track and the number of races...,"CREATE TABLE track (name VARCHAR, track_id VAR...",429,### Instruction:\n You are a powerful text-...
1,"SELECT T3.Shop_Name, T2.Carrier FROM stock AS ...",Show names of shops and the carriers of device...,"CREATE TABLE shop (Shop_Name VARCHAR, Shop_ID ...",2907,### Instruction:\n You are a powerful text-...


'### Instruction:\n    You are a powerful text-to-SQL model.     Your job is to answer questions about a database.     You are given a question and context regarding one or more tables.\n\n    You must output the SQL query that answers the question.\n\n    ### Input:\n    Find the locations that have more than one movie theater with capacity above 300.\n    ### Context:\n    CREATE TABLE cinema (LOCATION VARCHAR, capacity INTEGER)\n    ### Response:\n    '

In [18]:
import os

DATA_PATH = "/content/drive/MyDrive/Text2SQL/Data/"
PKL_DIR = "test/"
PKL_FILE = "sql_test.pkl"

full_path = os.path.join(DATA_PATH, PKL_DIR)
os.makedirs(full_path, exist_ok=True)

test_df.to_pickle(os.path.join(full_path, PKL_FILE))
print(f"File successfully saved at: {os.path.join(full_path, PKL_FILE)}")


File successfully saved at: /content/drive/MyDrive/Text2SQL/Data/test/sql_test.pkl


In [19]:
test_df.to_pickle(DATA_PATH + PKL_DIR + PKL_FILE)

In [22]:
test_df2=pd.read_pickle(DATA_PATH+'test/'+PKL_FILE)

#### Check Stored Test DataFrame

In [20]:
display(test_df.head(2))

,response,question,context,__index_level_0__,text
0,"SELECT T2.name, COUNT(*) FROM race AS T1 JOIN ...",Show the name of track and the number of races...,"CREATE TABLE track (name VARCHAR, track_id VAR...",429,### Instruction:\n You are a powerful text-...
1,"SELECT T3.Shop_Name, T2.Carrier FROM stock AS ...",Show names of shops and the carriers of device...,"CREATE TABLE shop (Shop_Name VARCHAR, Shop_ID ...",2907,### Instruction:\n You are a powerful text-...


In [23]:
display(test_df2.head(2))

,response,question,context,__index_level_0__,text
0,"SELECT T2.name, COUNT(*) FROM race AS T1 JOIN ...",Show the name of track and the number of races...,"CREATE TABLE track (name VARCHAR, track_id VAR...",429,### Instruction:\n You are a powerful text-...
1,"SELECT T3.Shop_Name, T2.Carrier FROM stock AS ...",Show names of shops and the carriers of device...,"CREATE TABLE shop (Shop_Name VARCHAR, Shop_ID ...",2907,### Instruction:\n You are a powerful text-...
